In [2]:
import pandas as pd
import numpy as np
import os,sys,time,datetime
from itertools import combinations
from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

start = time.time()

mod = 'holdout'

kfold = 5
ModelNumber = 500
#PairNumber = 100000
TopPair = 50000
ModelIndexList = [i for i in range(ModelNumber)]
PairIter = combinations(ModelIndexList, 2) # generate all combinations of pairs
PairList = list(PairIter)
SelectedPairs = PairList
end = time.time()
print('generate combinations done, combinations %s, time elapsed %.2fs' % (len(PairList), (end - start)))
#SelectedPairs = [PairList[i] for i in sorted(np.random.choice(range(len(PairList)), PairNumber, replace= False))]
#
meta_tree_mod = 'meta_trees_75_500'
BaseDir = '../../data/%s' % meta_tree_mod
InputDir = '%s/l1/kfold' % BaseDir
strategy = 'lgb_l2_%s' % meta_tree_mod

FoldData = []

for fold in range(kfold):
    data = pd.DataFrame()
    for t in range(ModelNumber):
        DataFile = '%s/%s/%s_%s_%s.csv' % (InputDir, fold, mod, strategy, t)
        tmp = pd.read_csv(DataFile, parse_dates= ['visit_date']).reset_index(drop= True)
        if(t == 0):
            data = tmp
        else:
            data['%s_%s' % (strategy, t)] = tmp['%s_%s' % (strategy, t)]
    FoldData.append(data)
    end = time.time()
    print('load fold %s data done, time elapsed %.2fs' % (fold, (end - start)))
end = time.time()
print('load data done, time elapsed %.2fs' % (end - start))
# print(FoldData[2].head(5))

PairMetrics = {}
for idx in range(len(SelectedPairs)):
    rmlse = .0
    col1 = '%s_%s' % (strategy, SelectedPairs[idx][0])
    col2 = '%s_%s' % (strategy, SelectedPairs[idx][1])
    for fold in range(kfold):
        rmlse += RMSLE((FoldData[fold][col1] + FoldData[fold][col2])/2, FoldData[fold]['visitors'])
    PairMetrics[SelectedPairs[idx]] = rmlse/kfold
    if(idx % 10000 == 0):
        end = time.time()
        print('calculating metric for %s done, time elapsed %.2fs' % (idx, (end - start)))

end = time.time()
print('calculating metric done, time elapsed %.2fs' % (end - start))
## sort by valid metrics
sorted_valid = sorted([(value,key) for (key,value) in PairMetrics.items()])

top_valid = [v[1] for v in sorted_valid[:TopPair]]

end = time.time()
print('sorting done, time elapsed %.2fs' % (end - start))
# save
OutputDir = '%s/l1/pairs' % BaseDir
if(os.path.exists(OutputDir) == False):
    os.makedirs(OutputDir)
ValidOutputFile = '%s/%s_top%s_%s.csv' % (OutputDir, meta_tree_mod, TopPair, mod)
with open(ValidOutputFile, 'w') as o_file:
    for v in top_valid:
        o_file.write('%s,%s\n' % (v[0], v[1]))
o_file.close()

generate combinations done, combinations 124750, time elapsed 0.02s
load fold 0 data done, time elapsed 19.38s
load fold 1 data done, time elapsed 38.94s
load fold 2 data done, time elapsed 58.49s
load fold 3 data done, time elapsed 77.32s
load fold 4 data done, time elapsed 96.07s
load data done, time elapsed 96.07s
calculating metric for 0 done, time elapsed 96.11s
calculating metric for 10000 done, time elapsed 134.16s
calculating metric for 20000 done, time elapsed 171.63s
calculating metric for 30000 done, time elapsed 209.03s
calculating metric for 40000 done, time elapsed 246.16s
calculating metric for 50000 done, time elapsed 283.71s
calculating metric for 60000 done, time elapsed 320.90s
calculating metric for 70000 done, time elapsed 358.40s
calculating metric for 80000 done, time elapsed 395.41s
calculating metric for 90000 done, time elapsed 432.56s
calculating metric for 100000 done, time elapsed 470.63s
calculating metric for 110000 done, time elapsed 508.51s
calculating 